In [1]:
import os
import pprint
from dotenv import load_dotenv
from typing import List, Optional
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import Qdrant
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langgraph.graph import START, END, StateGraph
from pydantic import BaseModel, Field
from qdrant_client import QdrantClient
from langchain_ollama import OllamaLLM
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import torch

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98;} </style>"))

In [3]:
# Cargar variables de entorno 
load_dotenv()
print("Variables de entorno cargadas")

Variables de entorno cargadas


In [4]:
# LLM
llm = OllamaLLM(model="hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:Q8_0")
print("LLM inicializado")
# ollama run hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:Q8_0

LLM inicializado


__Modelo de embedding Qwen/Qwen3-Embedding-0.6B__
https://huggingface.co/Qwen/Qwen3-Embedding-0.6B

In [27]:
torch.cuda.empty_cache()

In [28]:
device = 'cpu'
model_kwargs = {'device': device}

embeddings = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

2025-08-03 22:36:35,580 - INFO - Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-0.6B
2025-08-03 22:36:57,190 - INFO - 1 prompt is loaded, with the key: query


In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_kwargs = {'device': device}
encode_kwargs={"batch_size": 4}

embeddings = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

2025-08-03 22:33:05,769 - INFO - Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-0.6B


OutOfMemoryError: CUDA out of memory. Tried to allocate 594.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.56 GiB is allocated by PyTorch, and 92.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [29]:
# Configuración
COLLECTION_NAME = "idearq-qwen3"
VECTOR_SIZE = 1024

# Inicializar cliente
client = QdrantClient(url="http://localhost:6333")

# Borrar colección (si ya existe)
if COLLECTION_NAME in [c.name for c in client.get_collections().collections]:
    client.delete_collection(COLLECTION_NAME)

# Crear colección
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE),
)


2025-08-03 22:37:36,662 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-03 22:37:36,717 - INFO - HTTP Request: GET http://localhost:6333/collections "HTTP/1.1 200 OK"
2025-08-03 22:37:37,898 - INFO - HTTP Request: DELETE http://localhost:6333/collections/idearq-qwen3 "HTTP/1.1 200 OK"
C:\Users\elevi\AppData\Local\Temp\ipykernel_27228\1103362173.py:13: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
2025-08-03 22:37:37,948 - INFO - HTTP Request: DELETE http://localhost:6333/collections/idearq-qwen3 "HTTP/1.1 200 OK"
2025-08-03 22:37:39,554 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-qwen3 "HTTP/1.1 200 OK"


True

In [30]:
client = QdrantClient(url="http://localhost:6333")
info = client.get_collection(COLLECTION_NAME)
assert info.config.params.vectors.size == 1024, "La colección no está configurada para vectores de 1024 dimensiones."
print("¡La dimensión de los vectores (1024) es correcta!")

2025-08-03 22:37:39,822 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-03 22:37:39,876 - INFO - HTTP Request: GET http://localhost:6333/collections/idearq-qwen3 "HTTP/1.1 200 OK"


¡La dimensión de los vectores (1024) es correcta!


In [31]:
import logging
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

all_docs = []
folder_path = Path("pdf_articulos_idearq/") 

logging.info(f"Iniciando la carga de documentos desde: {folder_path.resolve()}")

if not folder_path.is_dir():
    logging.error(f"El directorio especificado no existe: {folder_path.resolve()}")
else:
    # Usamos .glob() que es eficiente para buscar archivos
    for pdf_file in folder_path.glob("*.pdf"):
        try:
            loader = PyMuPDFLoader(str(pdf_file), mode="single")
            docs_from_file = loader.load()
            all_docs.extend(docs_from_file)
            # logging.debug(f"Cargado: {pdf_file.name}") 
        except Exception as e:
            # Este bloque capturará si un PDF específico está dañado y no se puede leer
            logging.warning(f"No se pudo cargar el archivo '{pdf_file.name}'. Saltando. Error: {e}")

    logging.info(f"Carga finalizada. Se han cargado un total de {len(all_docs)} documentos.")

2025-08-03 22:37:40,278 - INFO - Iniciando la carga de documentos desde: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq
2025-08-03 22:38:49,456 - INFO - Carga finalizada. Se han cargado un total de 300 documentos.


In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configurar un logger para controlar el proceso
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

text_splitter_config = {
    "chunk_size": 1000,
    "chunk_overlap": 200, # 15-20% de chunk_size
    "length_function": len,
    "add_start_index": True,
    "separators": ["\n\n", "\n", ". ", " ", ""],
}

# El operador ** desempaqueta el diccionario como argumentos 
text_splitter = RecursiveCharacterTextSplitter(**text_splitter_config)

all_splits = text_splitter.split_documents(all_docs)

logging.info(f"Se dividieron {len(all_docs)} documentos en un total de {len(all_splits)} chunks.")

2025-08-03 22:38:59,679 - INFO - Se dividieron 300 documentos en un total de 50687 chunks.


In [33]:
# Test para ver la dimensión real del embedding
test_vec = embeddings.embed_query("test")
print("Dimensión real del embedding:", len(test_vec))

Dimensión real del embedding: 1024


In [34]:
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "idearq-qwen3"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Crear el objeto vector_store
client = QdrantClient(url=QDRANT_URL)

# Este objeto "envuelve" al cliente y sabe cómo usar la función de embeddings
logging.info(f"Conectando a la colección '{COLLECTION_NAME}' a través de LangChain...")
vector_store = Qdrant(
    client=client,
    collection_name=COLLECTION_NAME,
    embeddings=embeddings,
)
logging.info("Objeto 'vector_store' listo para la ingesta")

# Ingesta de chunks
logging.info(f"Iniciando la ingesta de {len(all_splits)} chunks...")

batch_size = 100  # valor ajustable según la gpu
for i in range(0, len(all_splits), batch_size):
    batch = all_splits[i:i+batch_size]
    vector_store.add_documents(batch)
    logging.info(f"Ingestados {i+len(batch)} de {len(all_splits)} chunks...")

logging.info("INGESTA COMPLETADA")

2025-08-03 22:39:04,548 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-03 22:39:04,563 - INFO - Conectando a la colección 'idearq-qwen3' a través de LangChain...
2025-08-03 22:39:04,603 - INFO - Objeto 'vector_store' listo para la ingesta
2025-08-03 22:39:04,604 - INFO - Iniciando la ingesta de 50687 chunks...
2025-08-03 22:41:36,252 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-qwen3/points?wait=true "HTTP/1.1 200 OK"
2025-08-03 22:42:48,006 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-qwen3/points?wait=true "HTTP/1.1 200 OK"
2025-08-03 22:42:48,058 - INFO - Ingestados 100 de 50687 chunks...
2025-08-03 22:45:02,275 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-qwen3/points?wait=true "HTTP/1.1 200 OK"
2025-08-03 22:45:59,952 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq-qwen3/points?wait=true "HTTP/1.1 200 OK"
2025-08-03 22:45:59,956 - INFO - Ingestados 200 de 50687 chun

KeyboardInterrupt: 

In [ ]:
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")

COLLECTION_NAME = "idearq-qwen3" 

print("\n--- Verificación en Qdrant ---")
print(f"Información de la colección '{COLLECTION_NAME}':")

# Usamos client.count() para obtener el número exacto de vectores
try:
    count_result = client.count(
        collection_name=COLLECTION_NAME, 
        exact=True
    )
    
    total_vectors = count_result.count
    
    if 'all_splits' in locals():
      print(f" - Chunks procesados en esta ejecución: {len(all_splits)}")

    print(f" - Número TOTAL de vectores en la colección: {total_vectors}")

except Exception as e:
    print(f"No se pudo obtener el conteo de la colección '{COLLECTION_NAME}'. Error: {e}")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = """
Eres un asistente experto en Prehistoria y Arqueología. Tu tarea es responder a la pregunta del usuario de forma clara y concisa, utilizando únicamente la información proporcionada en el siguiente contexto.

Contexto:
{context}

Pregunta: {query}

Instrucciones:
1. Lee la pregunta y el contexto cuidadosamente.
2. Sintetiza la información de los diferentes fragmentos del contexto para formular una respuesta completa.
3. Si el contexto contiene fechas o cronologías relevantes para la pregunta, asegúrate de incluirlas en tu respuesta.
4. Si el contexto no contiene información suficiente para responder a la pregunta, responde honestamente que no puedes encontrar la respuesta en la información proporcionada. No inventes información.

Respuesta:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
# Estado del grafo con Pydantic
class State(BaseModel):
    question: str
    context: List[Document] = Field(default_factory=list)
    answer: Optional[str] = None

# Nodo retrieve
def retrieve(state: State) -> dict:
    retrieved_docs = vector_store.similarity_search(state.question, k= 4)
    return {"context": retrieved_docs}

# Nodo generate
def generate(state: State) -> dict:
    docs_content = "\n\n".join(doc.page_content for doc in state.context)
    messages = prompt.invoke({"query": state.question, "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
result = graph.invoke({"question": "¿Cuáles son las dataciones más antiguas para el Neolítico Peninsular?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

2025-08-02 21:00:01,600 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-gte/points/search "HTTP/1.1 200 OK"
2025-08-02 21:00:45,085 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Acrobat Distiller 7.0 for Macintosh', 'creator': 'QuarkXPress: pictwpstops filter 1.0', 'creationdate': '2006-12-01T07:47:33+01:00', 'source': 'pdf_articulos_idearq\\616_2006.pdf', 'file_path': 'pdf_articulos_idearq\\616_2006.pdf', 'total_pages': 301, 'format': 'PDF 1.7', 'title': 'capitol 1-bo', 'author': 'Maria Pilar', 'subject': '', 'keywords': '', 'moddate': '2010-02-04T13:02:44+01:00', 'trapped': '', 'modDate': "D:20100204130244+01'00'", 'creationDate': "D:20061201074733+01'00'", 'start_index': 538555, '_id': 'b2506598-c5e2-4695-8f11-4c0fa442130e', '_collection_name': 'idearq-gte'}, page_content='que nos introduce directamente en la problemática de la expansión del Neolítico\na escala peninsular. \nEn la década de los años 90 del pasado s. XX los estudios sobre el Neolítico vivie-\nron una auténtica revolución empírica. Fruto de proyectos de investigación progra-\nmados e intervenciones de urgencia, se excavaron y dieron a conocer una ampl

In [ ]:
result = graph.invoke({"question": "¿Hay evidencias de caballo doméstico en el Calcolítico peninsular?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

2025-08-02 21:01:11,505 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-gte/points/search "HTTP/1.1 200 OK"
2025-08-02 21:01:14,510 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Acrobat Distiller 10.1.16 (Windows)', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2017-11-03T10:30:29+01:00', 'source': 'pdf_articulos_idearq\\993_Bartelheim_et_al_Iberian Chalcolithic_2017.pdf', 'file_path': 'pdf_articulos_idearq\\993_Bartelheim_et_al_Iberian Chalcolithic_2017.pdf', 'total_pages': 315, 'format': 'PDF 1.4', 'title': 'A9REA4E.tmp.pdf', 'author': 'Michelau', 'subject': '', 'keywords': '', 'moddate': '2017-11-03T10:30:29+01:00', 'trapped': '', 'modDate': "D:20171103103029+01'00'", 'creationDate': "D:20171103103029+01'00'", 'start_index': 330392, '_id': '915146c5-ed14-46f8-8918-1001ad4380e7', '_collection_name': 'idearq-gte'}, page_content='phase in terms of domestication processes, espe-\ncially with regard to equids, with different debates \nconcerning the existence of an Iberian refugium \nafter the Pleistocene, and the wild or domestic \nstatus of the horses in the 3rd mill. BC (Uerpmann \n1990; 1995; Benecke 19

In [ ]:
result = graph.invoke({"question": "¿Cuáles son las dataciones más antiguas para el Neolítico Peninsular?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

2025-08-02 21:01:42,250 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-gte/points/search "HTTP/1.1 200 OK"
2025-08-02 21:01:44,847 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Acrobat Distiller 7.0 for Macintosh', 'creator': 'QuarkXPress: pictwpstops filter 1.0', 'creationdate': '2006-12-01T07:47:33+01:00', 'source': 'pdf_articulos_idearq\\616_2006.pdf', 'file_path': 'pdf_articulos_idearq\\616_2006.pdf', 'total_pages': 301, 'format': 'PDF 1.7', 'title': 'capitol 1-bo', 'author': 'Maria Pilar', 'subject': '', 'keywords': '', 'moddate': '2010-02-04T13:02:44+01:00', 'trapped': '', 'modDate': "D:20100204130244+01'00'", 'creationDate': "D:20061201074733+01'00'", 'start_index': 538555, '_id': 'b2506598-c5e2-4695-8f11-4c0fa442130e', '_collection_name': 'idearq-gte'}, page_content='que nos introduce directamente en la problemática de la expansión del Neolítico\na escala peninsular. \nEn la década de los años 90 del pasado s. XX los estudios sobre el Neolítico vivie-\nron una auténtica revolución empírica. Fruto de proyectos de investigación progra-\nmados e intervenciones de urgencia, se excavaron y dieron a conocer una ampl

In [ ]:
result = graph.invoke({"question": "¿Cuál es la secuencia cultural de la Prehistoria y Protohistoria de Portugal?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

2025-08-02 21:01:55,053 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-gte/points/search "HTTP/1.1 200 OK"
2025-08-02 21:01:57,964 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'ilovepdf.com', 'creator': '', 'creationdate': '', 'source': 'pdf_articulos_idearq\\989_El-enclave-neolitico-al-aire-libre-del-Castillo-de-Dona-Mencia-Cordoba-Una-mirada-a-los-primeros-agricultores-y-ganaderos_2017.pdf', 'file_path': 'pdf_articulos_idearq\\989_El-enclave-neolitico-al-aire-libre-del-Castillo-de-Dona-Mencia-Cordoba-Una-mirada-a-los-primeros-agricultores-y-ganaderos_2017.pdf', 'total_pages': 28, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-06-11T11:42:27+02:00', 'trapped': '', 'modDate': "D:20180611114227+02'00'", 'creationDate': '', 'start_index': 30923, '_id': 'df20872d-736d-4e58-a2dc-2e226c221d33', '_collection_name': 'idearq-gte'}, page_content='sente e futuros, In: Diniz, M., Neves, C., Martins, A. (Eds.), O Neolítico em Portugal \nantes do Horizonte 2020: Perstectivas em debate. Associaçao dos Arqueólogos \nPortugueses, Lisboa, pp. 87-108.\nValente, M.J., Dean, R.M., Carvalho

In [ ]:
result = graph.invoke({"question": "¿Cuándo se produce el poblamiento de Baleares?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

2025-08-02 21:02:57,174 - INFO - HTTP Request: POST http://localhost:6333/collections/idearq-gte/points/search "HTTP/1.1 200 OK"
2025-08-02 21:02:59,995 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Context: [Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows); modified using iTextSharp™ 5.5.8 ©2000-2015 iText Group NV (AGPL-version)', 'creator': 'Servigistics Arbortext Advanced Print Publisher 11.1.4546/W-x64', 'creationdate': '2022-08-29T11:32:56+05:30', 'source': 'pdf_articulos_idearq\\1132_Caprine mobility_2021.pdf', 'file_path': 'pdf_articulos_idearq\\1132_Caprine mobility_2021.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': 'Caprine Mobility on the Balearic Islands During the Middle and Late Bronze Age (ca. 1600–850 BC): Fi', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2022-08-29T11:34:58+05:30', 'trapped': '', 'modDate': "D:20220829113458+05'30'", 'creationDate': "D:20220829113256+05'30'", 'start_index': 41771, '_id': '2e9bc357-1d3b-4b4c-af98-6e029c4146cc', '_collection_name': 'idearq-gte'}, page_content='“Cultura material, habitus, espacio y movilidad en el\narchipiélago balear durante el Bronce Final I (c. 1400–\n1100BC).” Bolletí de la S